# 🧳 Smart Travel Genie: AI-Powered Travel Itinerary Planner
A Capstone Project for the Gen AI Intensive Course 2025Q1 by OLIVIA TSENG (GitHub: @ollivvt)

## ✏️ Introduction

Planning a trip can be stressful, especially when you're trying to balance time, budget, and personal interests. This notebook demonstrates how generative AI can act as a personal travel assistant—automatically creating customized day-by-day itineraries based on a few user preferences.

By leveraging structured output generation, few-shot prompting, and retrieval-augmented logic, this project simulates how a large language model can replace hours of manual travel research. It's like having a travel-savvy friend who instantly knows where to go, what to eat, and what to do.

Tailored just for you!

## 🌍 Innovation and Impact

This project tackles a highly relatable real-world pain point: trip planning overload. Instead of searching dozens of blogs, review sites, and maps, the user simply inputs their destination, dates, and interests—and receives a structured, personalized itinerary in seconds.

This use case is:
- ✅ **Creative**: It transforms LLMs into virtual travel agents with a tone that adapts to user preferences.
- ✅ **Practical**: It can evolve into a real product with live data integration (e.g., Google Maps, weather, ticketing APIs).
- ✅ **Well-aligned with Gen AI**: It showcases three core capabilities:
  - **Structured Output**: The model returns a JSON-formatted, multi-day itinerary.
  - **Few-shot Prompting**: Examples guide the model to adopt specific styles (e.g., luxury vs. budget).
  - **RAG-style Simulation**: User interests steer the content selection, mimicking retrieval-augmented logic.


## 🔧 Setup: Install and Authenticate Google Gemini API

We use the `google-genai` Python package to connect to Google's Gemini Pro model. Gemini will be used to generate structured text content for each day's itinerary.

Note: In Kaggle, we use a private API key stored securely with `kaggle_secrets`.

In [1]:
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.2 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [4]:
# Confirm Gemini models available
for model in client.models.list():
    if "generateContent" in model.supported_actions:
        print("✅ Available model:", model.name)

✅ Available model: models/gemini-1.0-pro-vision-latest
✅ Available model: models/gemini-pro-vision
✅ Available model: models/gemini-1.5-pro-latest
✅ Available model: models/gemini-1.5-pro-001
✅ Available model: models/gemini-1.5-pro-002
✅ Available model: models/gemini-1.5-pro
✅ Available model: models/gemini-1.5-flash-latest
✅ Available model: models/gemini-1.5-flash-001
✅ Available model: models/gemini-1.5-flash-001-tuning
✅ Available model: models/gemini-1.5-flash
✅ Available model: models/gemini-1.5-flash-002
✅ Available model: models/gemini-1.5-flash-8b
✅ Available model: models/gemini-1.5-flash-8b-001
✅ Available model: models/gemini-1.5-flash-8b-latest
✅ Available model: models/gemini-1.5-flash-8b-exp-0827
✅ Available model: models/gemini-1.5-flash-8b-exp-0924
✅ Available model: models/gemini-2.5-pro-exp-03-25
✅ Available model: models/gemini-2.5-pro-preview-03-25
✅ Available model: models/gemini-2.0-flash-exp
✅ Available model: models/gemini-2.0-flash
✅ Available model: models/

## ✍️ Step 1: Define User Preferences

We simulate a user form where someone inputs their:
- Destination
- Travel dates
- Budget level
- Travel interests (e.g., food, temples, hiking)

These will guide the AI in generating a tailored itinerary.

In [5]:
user_input = {
    "destination": "Kyoto",
    "start_date": "2025-05-01",
    "end_date": "2025-05-07",
    "budget": "moderate",
    "interests": ["temples", "nature", "local food"]
}

## 🧠 Step 2: Few-Shot Prompting for Style Control

To help Gemini produce well-structured and stylistically appropriate responses, we provide a few-shot prompt. These examples teach the model what a luxury foodie itinerary or a budget backpacker itinerary might look like.

This technique improves consistency and makes the AI output more user-aligned.

In [6]:
few_shot_examples = [
    {
        "style": "luxury foodie",
        "output": """{
  "day": 1,
  "summary": "Arrival and indulgence in local gourmet",
  "morning": "Check into a 5-star ryokan in Gion.",
  "afternoon": "Explore Nishiki Market with a private food guide.",
  "evening": "Dinner at Michelin-starred Kikunoi."
}"""
    },
    {
        "style": "budget backpacker",
        "output": """{
  "day": 1,
  "summary": "Temple hopping on a shoestring",
  "morning": "Check into a hostel near Kyoto Station.",
  "afternoon": "Visit Fushimi Inari Shrine and hike up the trails.",
  "evening": "Grab dinner from a 7-11 and join a free walking tour."
}"""
    }
]

## 🛠️ Step 3: Building the Prompt

Here we dynamically construct a prompt for the Gemini model. The prompt includes:
- The user's input
- Several formatting examples (few-shot)
- Instructions to return results in structured JSON format

This is where we simulate grounding/RAG logic — by using user interests to implicitly guide what kind of activities the model should propose.

In [7]:
def build_prompt(user_input, few_shot_examples):
    prompt = (
        "You are a smart travel assistant that creates personalized travel itineraries in structured JSON format.\n\n"
        "Here are a few examples:\n"
    )
    for ex in few_shot_examples:
        prompt += f"# Style: {ex['style']}\n{ex['output']}\n\n"
    
    prompt += (
        "Now create an itinerary for the following user:\n"
        f"Destination: {user_input['destination']}\n"
        f"Dates: {user_input['start_date']} to {user_input['end_date']}\n"
        f"Budget: {user_input['budget']}\n"
        f"Interests: {', '.join(user_input['interests'])}\n\n"
        "Output the result as a list of JSON objects, one per day, each with keys: day, summary, morning, afternoon, evening."
    )
    return prompt

prompt = build_prompt(user_input, few_shot_examples)
print(prompt[:800])  # Preview first 800 characters of the prompt

You are a smart travel assistant that creates personalized travel itineraries in structured JSON format.

Here are a few examples:
# Style: luxury foodie
{
  "day": 1,
  "summary": "Arrival and indulgence in local gourmet",
  "morning": "Check into a 5-star ryokan in Gion.",
  "afternoon": "Explore Nishiki Market with a private food guide.",
  "evening": "Dinner at Michelin-starred Kikunoi."
}

# Style: budget backpacker
{
  "day": 1,
  "summary": "Temple hopping on a shoestring",
  "morning": "Check into a hostel near Kyoto Station.",
  "afternoon": "Visit Fushimi Inari Shrine and hike up the trails.",
  "evening": "Grab dinner from a 7-11 and join a free walking tour."
}

Now create an itinerary for the following user:
Destination: Kyoto
Dates: 2025-05-01 to 2025-05-07
Budget: moderate
I


## ✨ Step 4: Generate Itinerary with Gemini

We now pass the prompt to `gemini-2.0-flash` and ask it to generate a full itinerary. We're using Gemini's text model with temperature control to balance creativity and coherence.

The expected output is a multi-day itinerary formatted as JSON objects.

In [8]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=types.GenerateContentConfig(temperature=0.8),
)

generated_text = response.text
print("Gemini Output:\n", generated_text[:1000])

Gemini Output:
 ```json
[
  {
    "day": 1,
    "summary": "Arrival in Kyoto and Eastern Temple Exploration",
    "morning": "Arrive at Kyoto Station, check into a comfortable hotel near the station or in the Gion district.  Leave luggage and grab a quick ramen lunch nearby.",
    "afternoon": "Explore Kiyomizu-dera Temple. Take in the views and wander through the charming streets of Higashiyama district, sampling local sweets.",
    "evening": "Dinner in Gion, trying a traditional Kaiseki meal at a moderately priced restaurant (e.g., Gion Karyo). Afterwards, stroll along the Shirakawa Canal."
  },
  {
    "day": 2,
    "summary": "Golden Pavilion and Zen Garden Serenity",
    "morning": "Visit Kinkaku-ji (Golden Pavilion), taking in its stunning beauty reflected in the pond.",
    "afternoon": "Head to Ryoan-ji Temple and contemplate the famous Zen rock garden. Afterwards, explore the surrounding gardens and ponds.",
    "evening": "Enjoy a casual dinner at Nishiki Market, sampling va

## 🧼 Step 5: Parse Model Output into JSON

Gemini may sometimes return structured content with extra explanation or markdown wrappers. Here, we try to extract just the JSON list of itineraries for clean formatting and further use.

In [9]:
import json
import re

def extract_json_list(text):
    try:
        json_start = text.index("[")
        json_end = text.rindex("]") + 1
        json_str = text[json_start:json_end]
        return json.loads(json_str)
    except Exception as e:
        print("⚠️ Couldn't parse JSON list:", e)
        return None

itinerary_json = extract_json_list(generated_text)

## 📅 Step 6: Visualize the Final Travel Itinerary

We take the parsed JSON and display it in a user-friendly table. Each day includes:
- A summary
- Morning activity
- Afternoon activity
- Evening activity

The format allows for easy reading and could be adapted into apps or booking tools in future extensions.

In [10]:
import pandas as pd
from IPython.display import display, Markdown

def display_itinerary(itinerary_json):
    if itinerary_json and isinstance(itinerary_json, list):
        try:
            df = pd.DataFrame(itinerary_json)
            df.index = [f"Day {i+1}" for i in range(len(df))]  # Label rows
            styled_df = df.style.set_properties(
                **{
                    "text-align": "left",
                    "vertical-align": "top",
                    "white-space": "pre-wrap"
                }
            ).set_table_styles(
                [{'selector': 'th', 'props': [('text-align', 'left')]}]
            )
            display(Markdown("### 🗺️ Generated Itinerary"))
            display(styled_df)
        except Exception as e:
            print("⚠️ Failed to format itinerary table:", e)
    else:
        print("⚠️ No valid itinerary found or JSON was not a list.")

# Run display
display_itinerary(itinerary_json)


### 🗺️ Generated Itinerary

,day,summary,morning,afternoon,evening
Day 1,1,Arrival in Kyoto and Eastern Temple Exploration,"Arrive at Kyoto Station, check into a comfortable hotel near the station or in the Gion district. Leave luggage and grab a quick ramen lunch nearby.","Explore Kiyomizu-dera Temple. Take in the views and wander through the charming streets of Higashiyama district, sampling local sweets.","Dinner in Gion, trying a traditional Kaiseki meal at a moderately priced restaurant (e.g., Gion Karyo). Afterwards, stroll along the Shirakawa Canal."
Day 2,2,Golden Pavilion and Zen Garden Serenity,"Visit Kinkaku-ji (Golden Pavilion), taking in its stunning beauty reflected in the pond.","Head to Ryoan-ji Temple and contemplate the famous Zen rock garden. Afterwards, explore the surrounding gardens and ponds.","Enjoy a casual dinner at Nishiki Market, sampling various street foods and local delicacies. Try some Kyoto-style sushi."
Day 3,3,Arashiyama Bamboo Forest and Monkey Park,Take a train to Arashiyama. Explore the enchanting Bamboo Grove.,Visit the Arashiyama Monkey Park Iwatayama for panoramic views of Kyoto and playful interactions with monkeys (be prepared for a short hike).,"Have dinner in Arashiyama, enjoying a meal with a view of the Hozugawa River. Consider trying Yudofu (tofu hot pot), a local specialty."
Day 4,4,Fushimi Inari Shrine and Sake Tasting,"Visit Fushimi Inari Shrine, famous for its thousands of vibrant red torii gates. Hike as far up the mountain as you'd like.","Explore the Fushimi sake district. Visit a sake brewery for a tasting and learn about the sake-making process (e.g., Gekkeikan Okura Sake Museum).","Enjoy dinner at a local Izakaya (Japanese pub) near your hotel, trying various small dishes and local beers."
Day 5,5,Nara Day Trip: Deer and Todai-ji Temple,Take a short train ride to Nara. Immediately greet the friendly wild deer roaming Nara Park.,"Visit Todai-ji Temple, housing a giant bronze Buddha statue. Explore Kasuga Taisha Shrine with its thousands of lanterns.",Return to Kyoto and enjoy a casual dinner near Kyoto Station. Try Okonomiyaki (savory pancake).
Day 6,6,Imperial Palace and Nishijin Textile District,Visit the Kyoto Imperial Palace (advance booking may be required for guided tours). Explore the surrounding Kyoto Gyoen National Garden.,"Explore the Nishijin textile district, known for its traditional weaving techniques. Visit the Nishijin Textile Center for demonstrations and exhibits.","Enjoy a final Kyoto dinner at a restaurant specializing in Obanzai (Kyoto-style home cooking), experiencing a variety of seasonal vegetables and local flavors."
Day 7,7,Departure,Enjoy a final Japanese breakfast. Do some last-minute souvenir shopping at Kyoto Station.,"Depending on your flight/train schedule, you can visit Nijo Castle (former residence of the Tokugawa shogunate) for a couple of hours.",Depart from Kyoto.


## 🔍 Reflection and Future Work

This project demonstrates how Gen AI can quickly generate structured, personalized content. However, it is currently limited to static user inputs and simulated retrieval logic.

**Future improvements could include:**
- Integrating real APIs (Google Maps, Flights, Weather, Booking.com)
- Adding image generation to visualize destinations
- Creating a chatbot interface using Agents or Function Calling

This prototype shows how Gen AI can turn vague ideas into travel plans—instantly.